<a href="https://colab.research.google.com/github/AUT-Student/NLP-HW1/blob/main/NLP_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!gdown --id 16C0_9i0io43VfABV3-uukUjJYlM6k-2U
!unzip /content/HW1-datasets.zip

Downloading...
From: https://drive.google.com/uc?id=16C0_9i0io43VfABV3-uukUjJYlM6k-2U
To: /content/HW1-datasets.zip
100% 3.14M/3.14M [00:00<00:00, 195MB/s]
Archive:  /content/HW1-datasets.zip
   creating: content/HW1-datasets/
  inflating: content/HW1-datasets/train.txt  
  inflating: content/HW1-datasets/valid.txt  
  inflating: content/HW1-datasets/test_incomplete.txt  
  inflating: content/HW1-datasets/test.txt  
  inflating: content/HW1-datasets/test_incomplete_gold.txt  


In [12]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
file = open("/content/content/HW1-datasets/train.txt")

vocabulary = set()

for i, line in enumerate(file.readlines()):
  if i < 30*1000:
    words = line.split()

    for word in words:
      vocabulary.add(word)

In [5]:
len(vocabulary)

21188

In [6]:
sorted_vocabulary = sorted(vocabulary)

In [7]:
sorted_vocabulary.index('آب')

1

In [8]:
def convert_to_int(inputs):
  pass

In [13]:
class BigramNLM(keras.Model):
  def __init__(self, vocabulary):
    super().__init__()
    self.vocabulary = vocabulary
    self.model = keras.models.Sequential([
                                          keras.layers.Input((2,), name="Input"),
                                          keras.layers.Embedding(len(self.vocabulary), 128, name="Embedding"),
                                          keras.layers.Flatten(name="Flatten"),
                                          keras.layers.Dense(units=256, name="Hidden"),
                                          keras.layers.Dense(units=len(self.vocabulary), activation="softmax", name="Output"),
    ])

  def call(self, inputs):
    return self.model(inputs)

    # predicted_words = []

    # for input in inputs:
    #   word1, word2 = input[0], input[1]

    #   word1_index = self.vocabulary.index(word1)
    #   word2_index = self.vocabulary.index(word2)

    #   embedding_output1 = self.embedding(word1_index)
    #   embedding_output2 = self.embedding(word2_index)
    
    #   concat_output = tf.concat((embedding_output1, embedding_output2), 0)
      
    #   concat_output = tf.expand_dims(concat_output, 0)

    #   dense1_output = self.dense1(concat_output)
      
    #   dense2_output = self.dense2(dense1_output)

    #   dense2_output = tf.squeeze(dense2_output)

    #   predicted_word_index = tf.argmax(dense2_output)
    #   predicted_word = self.vocabulary[predicted_word_index]
    #   predicted_words.append(predicted_word)

    # return tf.convert_to_tensor(predicted_words)

In [14]:
bnlm = BigramNLM(vocabulary=sorted_vocabulary)

In [15]:
bnlm.call(tf.constant([
                        [1, 2],
                        [3, 4],
                        [1, 2]
                        ]))

<tf.Tensor: shape=(3, 21188), dtype=float32, numpy=
array([[4.7299269e-05, 4.7201942e-05, 4.7236957e-05, ..., 4.7213598e-05,
        4.7258622e-05, 4.7380388e-05],
       [4.7402002e-05, 4.7186091e-05, 4.7095156e-05, ..., 4.7112935e-05,
        4.7582958e-05, 4.7419049e-05],
       [4.7299269e-05, 4.7201942e-05, 4.7236957e-05, ..., 4.7213598e-05,
        4.7258622e-05, 4.7380388e-05]], dtype=float32)>

In [17]:
np.argmax(bnlm.call(tf.constant([
                                  [1, 2],
                                  [3, 4],
                                  [1, 2]
                                  ])), axis=1)

array([13748, 18117, 13748])

In [95]:
embedding_test(tf.constant([[1, 2, 1], [1, 2, 4]]))

<tf.Tensor: shape=(2, 3, 4), dtype=float32, numpy=
array([[[-0.03383321, -0.00586923,  0.03602019,  0.04541481],
        [ 0.0401669 , -0.03896043,  0.02758982,  0.03695711],
        [-0.03383321, -0.00586923,  0.03602019,  0.04541481]],

       [[-0.03383321, -0.00586923,  0.03602019,  0.04541481],
        [ 0.0401669 , -0.03896043,  0.02758982,  0.03695711],
        [-0.00159363, -0.00924385, -0.00259749, -0.02089017]]],
      dtype=float32)>

In [90]:
bnlm.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [87]:

bnlm.call(word1 = "من", word2 = "رفتم")

word1_index = 15312
word2_index = 8714
embedding shape = [128]
concat shape = [  1 256]
dense1_output shape = [  1 256]
dense2_output shape = [21188]
dense2 max = 4.794905544258654e-05
dense2 shape = [21188]
predicted_word_index = 21112
predicted_word = یه


'یه'

In [ ]:
tensor1 = tf.constant([1,2,3])

In [47]:
tf.expand_dims(tensor1, 0)

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[1, 2, 3]], dtype=int32)>

In [52]:
tf.keras.backend.get_value(max(tensor1))

3

In [68]:
tf.argmax(tensor1)

<tf.Tensor: shape=(), dtype=int64, numpy=2>

In [109]:
flatten_test = keras.layers.Flatten()

In [111]:
flatten_test(tf.constant([
                          [[1,2,3], [2,3,4]],
                          [[4,5,3], [7,3,4]],
                          ]))

<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[1, 2, 3, 2, 3, 4],
       [4, 5, 3, 7, 3, 4]], dtype=int32)>

In [134]:
keras.backend.argmax(tf.constant([11,2,3,5,3]))

<tf.Tensor: shape=(), dtype=int64, numpy=0>